In [ ]:
import graphics
import time
import random
import math

CANVAS_WIDTH = 500
CANVAS_HEIGHT = 600
PADDLE_Y = CANVAS_HEIGHT - 30
PADDLE_WIDTH = 80
PADDLE_HEIGHT = 15
BALL_RADIUS = 10

BRICK_GAP = 5
BRICK_WIDTH = (CANVAS_WIDTH - BRICK_GAP * 9) / 10 # 45.5
BRICK_HEIGHT = 10


DIST_FROM_TOP = CANVAS_HEIGHT / 10
BALL_DIAMETER = BALL_RADIUS * 2
DELAY = 0.01


def create_bricks(canvas):
    
    
    
    bricks = []
    for row in range(10):
        for col in range(10):
            
            left_x = col * (BRICK_WIDTH + BRICK_GAP)
            right_x = left_x + BRICK_WIDTH
    
            top_y = row * (BRICK_HEIGHT + BRICK_GAP) + DIST_FROM_TOP
            bottom_y = top_y + BRICK_HEIGHT
            
            if row < 2:
                color = "red"
            elif row == 2 or row == 3:
                color ="orange"
            elif row == 4 or row == 5:
                color = "yellow"
            elif row == 6 or row == 7:
                color = "green"
            elif row == 8 or row == 9:
                color = "cyan"
                
            brick = canvas.create_rectangle(left_x, top_y, right_x, bottom_y, color)
            bricks.append(brick)
            
    return bricks
    

def create_ball(canvas):
    left_x = CANVAS_WIDTH / 2 - BALL_RADIUS
    right_x = left_x + BALL_DIAMETER
    
    top_y = CANVAS_HEIGHT / 2 - BALL_RADIUS
    bottom_y = top_y + BALL_DIAMETER
    
    return canvas.create_oval(left_x, top_y, right_x, bottom_y, "blue")
    

def create_paddle(canvas):
    
    left_x = (CANVAS_WIDTH / 2) - (PADDLE_WIDTH / 2)
    right_x = left_x + PADDLE_WIDTH
    
    top_y = PADDLE_Y
    bottom_y = top_y + PADDLE_HEIGHT
    
    return canvas.create_rectangle(left_x, top_y, right_x, bottom_y)


def main():
    canvas = graphics.Canvas(CANVAS_WIDTH,CANVAS_HEIGHT)
    
    bricks = create_bricks(canvas)
    ball = create_ball(canvas)
    paddle = create_paddle(canvas)
    
    bricks_len = len(bricks)
    turns = 3
    

    # initial object motion and positions
    ball_change_x = 5
    ball_change_x = ball_change_x if random.random() < 0.5 else -ball_change_x
    ball_change_y = 10
    
    # animation loop
    while turns > 0 and bricks_len > 0:
        # update location of ball and wait
        canvas.move(ball, ball_change_x, ball_change_y)
        time.sleep(DELAY)
        
        ball_left_x = canvas.get_left_x(ball)
        ball_top_y = canvas.get_top_y(ball)
        
        # lose try and reset ball to middle
        if ball_top_y >= CANVAS_HEIGHT:
            turns -= 1
            canvas.moveto(ball, CANVAS_WIDTH / 2 - BALL_RADIUS, CANVAS_HEIGHT / 2 - BALL_RADIUS)
        
        # collision with canvas
        if ball_left_x <= 0 or ball_left_x + BALL_DIAMETER >= CANVAS_WIDTH:
            ball_change_x = -ball_change_x
        
        
        
        if ball_top_y <= 0:
            ball_change_y = -ball_change_y    

        # paddle follow mouse on x-axis
        mouse_x = canvas.get_mouse_x()
        canvas.moveto(paddle, mouse_x - (PADDLE_WIDTH / 2), PADDLE_Y)
        
        # get object collisions in ball "bounding box"
        ball_right_x = ball_left_x + BALL_DIAMETER
        ball_bottom_y = ball_top_y + BALL_DIAMETER
       
        colliding_list = canvas.find_overlapping(ball_left_x, ball_top_y, ball_right_x, ball_bottom_y)
        
        # ball collided with the paddle
        if paddle in colliding_list:
            ball_change_y = -ball_change_y
            canvas.moveto(ball, ball_left_x, canvas.get_top_y(paddle) - abs(ball_change_y))
            
        
        # ball collided with brick
        if any(brick in colliding_list for brick in bricks):
            ball_change_y = -ball_change_y   
        
        
        
        
        
        # remove one brick that was collided with
        for obj in colliding_list:
            if obj != paddle and obj != ball:
                canvas.delete(obj)
                bricks_len -= 1
                break
    
    canvas.delete(ball)
    
    # print win or lose
    message, color = ("Congratulations! You won!", "green") if bricks_len <= 0 else ("Game Over", "red")
    font_size = 20
    offset = (len(message) * font_size) / 4
    canvas.create_text(CANVAS_WIDTH / 2 - offset, CANVAS_HEIGHT / 2, text = message, font_size = font_size, color = color)


if __name__ == '__main__':
    main()